In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv1D, MaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Assuming you have loaded your ECG dataset and labels
# X, y = load_ecg_data()

# Convert labels to one-hot encoding
num_classes = 5  # Replace with your actual number of classes
y = to_categorical(y, num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the base ResNet-50 model
def create_resnet50(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the weights of the pre-trained layers

    model = Sequential([
        base_model,
        GlobalAveragePooling1D(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    return model

# Create the ResNet-50 model for your specific input shape and number of classes
input_shape = X_train.shape[1:]  # Shape of one ECG signal
resnet50_model = create_resnet50(input_shape, num_classes)

# Compile the model with Adam optimizer, a learning rate of 0.01, and categorical crossentropy loss
adam_optimizer = Adam(lr=0.01)
resnet50_model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with a batch size of 128
resnet50_model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

# Evaluate the model
accuracy = resnet50_model.evaluate(X_test, y_test)[1]
print(f"Test Accuracy: {accuracy * 100:.2f}%")
